In [1]:
from rocketcea.cea_obj_w_units import CEA_Obj
from rocketcea.cea_obj import add_new_fuel
import numpy as np
from scipy.optimize import minimize

In [2]:
allvectors = []               # array for all design vecs, global variable
allobjfun = []                # array for tracking objective function evaluations

In [3]:
def objective(var, cons):
    return (var/cons)**2 / 2
def exterior(var, cons, good_if_less_than=False):
    if good_if_less_than:
        return np.max([0, var/cons - 1])**2 / 2
    else:
        return np.max([0, -(var/cons - 1)])**2 / 2
def proportion(amount, ratio):
    top = amount * ratio/(1 + ratio)
    bottom = amount * 1/(1 + ratio)
    return top, bottom

In [4]:
def get_propellant_properties(alc_wt, of_ratio, p_ch, exp_ratio):
    ipa_wt = min(alc_wt, 99)
    eth_ratio = 21.11
    alc_wt = min(alc_wt, 99.5)
    eth_wt1, eth_wt2 = proportion(alc_wt, eth_ratio)
    ipa_str = '''
    fuel C3H8O-2propanol C 3 H 8 O 1    wt%=''' + str(ipa_wt) + '''
    h,cal=-65133.     t(k)=298.15   rho,g/cc=0.786
    fuel water H 2 O 1  wt%=''' + str(100 - ipa_wt) + '''
    h,cal=-68308.  t(k)=298.15 rho,g/cc=0.9998
    '''
    eth_str = '''
    fuel C2H5OH(L)   C 2 H 6 O 1       wt%=''' + str(eth_wt1) + '''
    h,cal=-66370.0      t(k)=298.15       rho,g/cc=0.789
    fuel C3H8O-2propanol C 3 H 8 O 1    wt%=''' + str(eth_wt2) + '''
    h,cal=-65133.     t(k)=298.15   rho,g/cc=0.786
    fuel water H 2 O 1  wt%=''' + str(100 - alc_wt) + '''
    h,cal=-68308.  t(k)=298.15 rho,g/cc=0.9998
    '''
    add_new_fuel('LV4_Fuel', ipa_str)
    #add_new_fuel('LV4_Fuel', eth_str)
    PROPELLANT = CEA_Obj(oxName='LOX', fuelName='LV4_Fuel',
                        pressure_units='Pa', temperature_units='K', cstar_units='m/s',
                         density_units='kg/m^3', isp_units='sec', specific_heat_units='J/kg-K')
    isp = PROPELLANT.get_Isp(Pc=p_ch, MR=of_ratio, eps=exp_ratio)
    IspVac, Cstar, Tc, MW, gamma = PROPELLANT.get_IvacCstrTc_ChmMwGam(Pc=p_ch, MR=of_ratio, eps=exp_ratio)
    return [isp, IspVac, Cstar, Tc, MW, gamma]

def prop_cost(x):
    ipa_wt, of_ratio, p_ch, exp_ratio = x
    ipa_wt = 64.8
    isp, IspVac, Cstar, Tc, MW, gamma = get_propellant_properties(ipa_wt, of_ratio, p_ch, exp_ratio)
    #print(-objective(IspVac, 230), 50 * exterior(p_ch, 2413166, True) , 50 * exterior(exp_ratio, 5.988, True))
    merit = -objective(IspVac, 230) + 50* exterior(p_ch, 2413166, True) + 50 * exterior(exp_ratio, 5.988, True)
    return merit

def prop_opt(x0):
    res = minimize(prop_cost, x0, method='nelder-mead', options={'disp': True, 'adaptive':True, 'xatol': 0.01, 'fatol': 0.01})
    x = res.x # feed optimal design vec into next iteration
    return x

def prop_run():
    x = prop_opt([64.8, 1.3, 2413166, 5.988])
    #x = [64.8, 1.3, 2413166, 5.988]
    ipa_wt, of_ratio, p_ch, exp_ratio = x
    ipa_wt = min(ipa_wt, 99)
    ipa_wt = 64.8
    isp, IspVac, Cstar, Tc, MW, gamma = get_propellant_properties(ipa_wt, of_ratio, p_ch, exp_ratio)
    if __name__ == '__main__' and not '__file__' in globals():
        print('Alcohol Wt %: ', ipa_wt)
        print('OF ratio: ', of_ratio)
        print('P_ch (Pa): ', p_ch)
        print('Expansion ratio: ', exp_ratio)
        print()
        print('Vacuum ISP (s): ', IspVac)
        print('Chamber Temp (K): ', Tc)
        print('Molar Wt (1/n): ', MW)
        print('Spec Heat: ', gamma)
    global PROPELLANT_SET
    PROPELLANT_SET = True
    return ipa_wt, of_ratio, p_ch, Tc, MW, gamma
    

In [5]:
if __name__ == '__main__' and not '__file__' in globals(): prop_run()

reading cea isp data files for LOX / LV4_Fuel 100 times
reading cea isp data files for LOX / LV4_Fuel 200 times
reading cea isp data files for LOX / LV4_Fuel 300 times
Optimization terminated successfully.
         Current function value: -0.779160
         Iterations: 182
         Function evaluations: 391
Alcohol Wt %:  64.8
OF ratio:  1.3846692240985337
P_ch (Pa):  2413666.298058278
Expansion ratio:  6.007040500102594

Vacuum ISP (s):  287.1619484205367
Chamber Temp (K):  3106.1230925802856
Molar Wt (1/n):  23.725017448089833
Spec Heat:  1.1214413773331058


In [6]:
if __name__ == '__main__' and not '__file__' in globals(): prop_run()

reading cea isp data files for LOX / LV4_Fuel 400 times
reading cea isp data files for LOX / LV4_Fuel 500 times
reading cea isp data files for LOX / LV4_Fuel 600 times
reading cea isp data files for LOX / LV4_Fuel 700 times
Optimization terminated successfully.
         Current function value: -0.779160
         Iterations: 182
         Function evaluations: 391
Alcohol Wt %:  64.8
OF ratio:  1.3846692240985337
P_ch (Pa):  2413666.298058278
Expansion ratio:  6.007040500102594

Vacuum ISP (s):  287.1619484205367
Chamber Temp (K):  3106.1230925802856
Molar Wt (1/n):  23.725017448089833
Spec Heat:  1.1214413773331058
